In [1]:
# target_snp.py

"""Program to create targets form sized pickles
Date: 23-July-2019
Ver: 1.0
Time taken: 120 milliseconds
"""

from z_helper import *
util.startLoop()

# from json
a = assign_var('common') + assign_var('snp')
for v in a:
    exec(v)

def target_snp(ib, df_sized, blacklist):
    '''Args:
        (ib) as connection object
        (df_sized) picked up from sized pickle
        (blacklist) as list of blacklisted stocks
       Returns:
        tb: trade blocks and pickles into targets.pickle
    '''
    
    # from portfolio
    #_______________
#     with get_connected('snp', 'live') as ib:
    p = util.df(ib.portfolio()) # portfolio table

    # extract option contract info from portfolio table
    dfp = pd.concat([p, util.df([c for c in p.contract])[util.df([c for c in p.contract]).columns[:7]]], axis=1).iloc[:, 1:]
    dfp = dfp.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

    # get the total position
    dfp1 = dfp.groupby('symbol').sum()['position']

    # from options pickle
    #____________________

    # get the options
#     df_opt = pd.read_pickle(fspath+'sized_snp.pkl')
    df_opt = df_sized.assign(und_remq=(snp_assignment_limit/(df_opt.lot*df_opt.undPrice)).astype('int')) # remaining quantities in entire snp

    # remove nan in margins and close prices. These could be dead ones.
    df_opt = df_opt[~df_opt.margin.isnull()].reset_index(drop=True)
    df_opt = df_opt[~df_opt.close.isnull()].reset_index(drop=True)

    # remove margins with 1e7
    df_opt = df_opt[df_opt.margin < 1e7]

    # establish quantity and minimum expected price and sdmult
    df_opt = df_opt.assign(qty=1, expPrice = df_opt.close+0.1, sd=abs(df_opt.strike-df_opt.undPrice)/df_opt.stDev)

    # recacluate rom based on expPrice upgrade
    df_opt = df_opt.assign(rom=df_opt.expPrice/df_opt.margin*365/df_opt.dte*df_opt.lot)

    # sort the standard deviation change (lowest sd is most risky)
    df_opt = df_opt.sort_values('sd', ascending=True)

    # for those not meeting minimum expected ROM, up the expected price
    rom_mask = (df_opt.rom < minexpRom)
    df_opt.loc[rom_mask, 'expPrice'] = ((df_opt[rom_mask].expPrice * minexpRom )/ df_opt[rom_mask].rom).apply(lambda x: get_prec(x, prec))

    # filter based on remaining quantity
    #___________________________________

    # compute the remaining quantities
    df_opt1 = df_opt.groupby('symbol').first()[['lot', 'margin', 'und_remq']]

    df_opt2 = df_opt1.join(dfp1).fillna(0).astype('int')
    df_opt2 = df_opt2.assign(remqty=df_opt2.und_remq+(df_opt2.position/df_opt2.lot).astype('int'))

    dfrq = df_opt2[['remqty']]

    # remove existing positions with remqty < 1
    blacklist = blacklist + list(dfrq[dfrq.remqty < 1].index)
    df_opt = df_opt[~df_opt.symbol.isin(blacklist)]

    # remove targets with expPrice 0.0. This is caused by negative margins
    df_opt.loc[df_opt.expPrice < minexpOptPrice, 'expPrice'] = minexpOptPrice

    # pd.options.display.float_format = '{:,.2f}'.format
    cols = ['symbol', 'undId', 'optId', 'dte', 'lot', 'right', 'undPrice', 'strike', 'stDev', 'lo52', 'hi52', 'margin', 'qty', 'close', 'rom', 'sd', 'expPrice']
    df_opt[cols].to_pickle(fspath+'targets.pkl')
    
    return df_opt[cols]

In [ ]:
%%time

##### script ignored by jup2py

from z_helper import *
util.startLoop()

# from json
a = assign_var('common') + assign_var('snp')
for v in a:
    exec(v)

# from portfolio
#_______________
with get_connected('snp', 'live') as ib:
    p = util.df(ib.portfolio()) # portfolio table

# extract option contract info from portfolio table
dfp = pd.concat([p, util.df([c for c in p.contract])[util.df([c for c in p.contract]).columns[:7]]], axis=1).iloc[:, 1:]
dfp = dfp.rename(columns={'lastTradeDateOrContractMonth': 'expiration'})

# get the total position
dfp1 = dfp.groupby('symbol').sum()['position']

# from options pickle
#____________________

# get the options
df_opt = pd.read_pickle(fspath+'sized_snp.pkl')
df_opt = df_opt.assign(und_remq=(snp_assignment_limit/(df_opt.lot*df_opt.undPrice)).astype('int')) # remaining quantities in entire snp

# remove nan in margins and close prices. These could be dead ones.
df_opt = df_opt[~df_opt.margin.isnull()].reset_index(drop=True)
df_opt = df_opt[~df_opt.close.isnull()].reset_index(drop=True)

# remove margins with 1e7
df_opt = df_opt[df_opt.margin < 1e7]

# establish quantity and minimum expected price and sdmult
df_opt = df_opt.assign(qty=1, expPrice = df_opt.close+0.1, sd=abs(df_opt.strike-df_opt.undPrice)/df_opt.stDev)

# recacluate rom based on expPrice upgrade
df_opt = df_opt.assign(rom=df_opt.expPrice/df_opt.margin*365/df_opt.dte*df_opt.lot)

# sort the standard deviation change (lowest sd is most risky)
df_opt = df_opt.sort_values('sd', ascending=True)

# for those not meeting minimum expected ROM, up the expected price
rom_mask = (df_opt.rom < minexpRom)
df_opt.loc[rom_mask, 'expPrice'] = ((df_opt[rom_mask].expPrice * minexpRom )/ df_opt[rom_mask].rom).apply(lambda x: get_prec(x, prec))

# filter based on remaining quantity
#___________________________________

# compute the remaining quantities
df_opt1 = df_opt.groupby('symbol').first()[['lot', 'margin', 'und_remq']]

df_opt2 = df_opt1.join(dfp1).fillna(0).astype('int')
df_opt2 = df_opt2.assign(remqty=df_opt2.und_remq+(df_opt2.position/df_opt2.lot).astype('int'))

dfrq = df_opt2[['remqty']]

# remove existing positions with remqty < 1
blacklist = blacklist + list(dfrq[dfrq.remqty < 1].index)
df_opt = df_opt[~df_opt.symbol.isin(blacklist)]

# remove targets with expPrice 0.0. This is caused by negative margins
df_opt.loc[df_opt.expPrice < minexpOptPrice, 'expPrice'] = minexpOptPrice

# pd.options.display.float_format = '{:,.2f}'.format
cols = ['symbol', 'undId', 'optId', 'dte', 'lot', 'right', 'undPrice', 'strike', 'stDev', 'lo52', 'hi52', 'margin', 'qty', 'close', 'rom', 'sd', 'expPrice']
df_opt[cols].to_pickle(fspath+'targets.pkl')

In [ ]:
# test target_snp

df_opt=pd.read_pickle(fspath+'sized_snp.pkl')
with get_connected('snp', 'live') as ib:
    df = target_snp(ib, df_opt, blacklist)

In [5]:
df.head()

,symbol,undId,optId,dte,lot,right,undPrice,strike,stDev,lo52,hi52,margin,qty,close,rom,sd,expPrice
5,AGN,196610642,346887908,24,100,P,162.63,120.0,17.241693,115.73,193.46,769.64,1,0.05,0.296405,2.472495,0.35
543,SBUX,274105,373263845,59,100,P,90.92,77.5,4.830548,48.54,91.50,1010.22,1,0.34,0.269450,2.778153,1.14
1133,MDT,181387075,357015760,59,100,C,101.24,115.0,4.773272,76.55,101.34,1359.32,1,0.14,0.109227,2.882719,1.54
1001,JNJ,8719,370971895,10,100,C,128.64,142.0,4.555232,119.40,147.84,2280.87,1,0.00,0.160027,2.932891,0.44
1238,TGT,6437,370783699,59,100,C,87.22,105.0,5.980987,61.13,89.26,528.64,1,0.10,0.234051,2.972754,0.60
